In [8]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import Row, SQLContext
from pyspark import SparkFiles
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import os
import pandas as pd


In [9]:
#######################
##CREATE SPARK CONTEXT#
##CREATE SQL CONTEXT###
#######################
sc =SparkContext()
sqlContext = SQLContext(sc)


In [10]:
#LOAD IRIS DATA

data_dir="."
file = os.path.join(data_dir,"iris.csv")
panda_df = pd.read_csv(file)

iris_df=sqlContext.createDataFrame(panda_df)
iris_df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



#### Perform Data Analytics

In [11]:
#See standard parameters
iris_df.describe().show()

+-------+------------------+------------------+------------------+------------------+---------+
|summary|      sepal_length|       sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+------------------+------------------+------------------+---------+
|  count|               150|               150|               150|               150|      150|
|   mean| 5.843333333333332|3.0573333333333337| 3.758000000000001|1.1993333333333331|     null|
| stddev|0.8280661279778628|0.4358662849366982|1.7652982332594664|0.7622376689603467|     null|
|    min|               4.3|               2.0|               1.0|               0.1|   Setosa|
|    max|               7.9|               4.4|               6.9|               2.5|Virginica|
+-------+------------------+------------------+------------------+------------------+---------+



### Decision tree Classifier

#### Prepare data for ML

In [12]:
#Transform to a Data Frame for input to Machine Learing

#stage 1: change the categorical feature to numerical one 
stringIndexer = StringIndexer(inputCol="variety", outputCol="label")


#stage 2: generate one column "features" for all the features 
vectorAssembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"],\
                                  outputCol="features")

#stage 3: create the model 
classifier_dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

#regroup the stages in a pipeline
pipeline_dt = Pipeline(stages=[stringIndexer, vectorAssembler, classifier_dt])

# split data
(data_train, data_test) = iris_df.randomSplit([0.75, 0.25])

#### Perform Machine Learning

In [13]:
# apply the pipeline to apply the transformation and train the model 
model_dt = pipeline_dt.fit(data_train)

# make predictions by transforming the features dataframe into predictions dataframe
predictions_dt = model_dt.transform(data_test)

print("nb nodes in model dt: ",model_dt.stages[-1].numNodes)
print("depth of model dt: ",model_dt.stages[-1].depth)

predictions_dt.select("prediction","label", "features").show()

#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
accuracy_dt = evaluator.evaluate(predictions_dt)    

print("Test accuracy = ", accuracy_dt*100, "%")

#Draw a confusion matrix
predictions_dt.groupBy("label","prediction").count().show()

nb nodes in model dt:  17
depth of model dt:  5
+----------+-----+-----------------+
|prediction|label|         features|
+----------+-----+-----------------+
|       2.0|  2.0|[4.4,2.9,1.4,0.2]|
|       2.0|  2.0|[4.6,3.4,1.4,0.3]|
|       2.0|  2.0|[4.8,3.0,1.4,0.1]|
|       2.0|  2.0|[4.8,3.1,1.6,0.2]|
|       2.0|  2.0|[4.9,3.6,1.4,0.1]|
|       2.0|  2.0|[5.0,3.3,1.4,0.2]|
|       2.0|  2.0|[5.1,3.3,1.7,0.5]|
|       2.0|  2.0|[5.1,3.5,1.4,0.2]|
|       2.0|  2.0|[5.1,3.8,1.5,0.3]|
|       2.0|  2.0|[5.1,3.8,1.6,0.2]|
|       2.0|  2.0|[5.2,3.4,1.4,0.2]|
|       2.0|  2.0|[5.4,3.4,1.7,0.2]|
|       2.0|  2.0|[5.4,3.9,1.3,0.4]|
|       0.0|  0.0|[5.5,2.3,4.0,1.3]|
|       2.0|  2.0|[5.5,4.2,1.4,0.2]|
|       0.0|  0.0|[5.6,2.5,3.9,1.1]|
|       0.0|  0.0|[5.7,2.8,4.5,1.3]|
|       0.0|  0.0|[6.0,2.2,4.0,1.0]|
|       0.0|  0.0|[6.1,2.8,4.7,1.2]|
|       0.0|  0.0|[6.2,2.2,4.5,1.5]|
+----------+-----+-----------------+
only showing top 20 rows

Test accuracy =  100.0 %
+-----+------

### Random Forest Classifier

#### Prepare data for ML

In [14]:
#Transform to a Data Frame for input to Machine Learing

#stage 1: as before
#stage 2: as before 
                                  
#stage 3: create the model 
classifier_rf = RandomForestClassifier(labelCol="label", featuresCol="features")

#regroup the stages in a pipeline
pipeline_rf = Pipeline(stages=[stringIndexer, vectorAssembler, classifier_rf])

# split data
#(data_train, data_test) = iris_df.randomSplit([0.75, 0.25])

#### Perform Machine Learning

In [15]:
# apply the pipeline to apply the transformation and train the model 
model_rf = pipeline_rf.fit(data_train)

# make predictions by transforming the features dataframe into predictions dataframe
predictions_rf = model_rf.transform(data_test)

print("model info : ",model_rf.stages[-1])

predictions_rf.select("prediction","label", "features").show()

#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
accuracy_rf = evaluator.evaluate(predictions_rf)    

print("Test accuracy = ", accuracy_rf*100, "%")

#Draw a confusion matrix
predictions_rf.groupBy("label","prediction").count().show()

model info :  RandomForestClassificationModel: uid=RandomForestClassifier_bd811ce424c1, numTrees=20, numClasses=3, numFeatures=4
+----------+-----+-----------------+
|prediction|label|         features|
+----------+-----+-----------------+
|       2.0|  2.0|[4.4,2.9,1.4,0.2]|
|       2.0|  2.0|[4.6,3.4,1.4,0.3]|
|       2.0|  2.0|[4.8,3.0,1.4,0.1]|
|       2.0|  2.0|[4.8,3.1,1.6,0.2]|
|       2.0|  2.0|[4.9,3.6,1.4,0.1]|
|       2.0|  2.0|[5.0,3.3,1.4,0.2]|
|       2.0|  2.0|[5.1,3.3,1.7,0.5]|
|       2.0|  2.0|[5.1,3.5,1.4,0.2]|
|       2.0|  2.0|[5.1,3.8,1.5,0.3]|
|       2.0|  2.0|[5.1,3.8,1.6,0.2]|
|       2.0|  2.0|[5.2,3.4,1.4,0.2]|
|       2.0|  2.0|[5.4,3.4,1.7,0.2]|
|       2.0|  2.0|[5.4,3.9,1.3,0.4]|
|       0.0|  0.0|[5.5,2.3,4.0,1.3]|
|       2.0|  2.0|[5.5,4.2,1.4,0.2]|
|       0.0|  0.0|[5.6,2.5,3.9,1.1]|
|       0.0|  0.0|[5.7,2.8,4.5,1.3]|
|       0.0|  0.0|[6.0,2.2,4.0,1.0]|
|       0.0|  0.0|[6.1,2.8,4.7,1.2]|
|       0.0|  0.0|[6.2,2.2,4.5,1.5]|
+----------+-----+--

### LogisticRegression Classifier 

Unfortunatly Pyspark does not handle multi class problems for the gradient boosting. I'll use the logistic regression instead.

#### Prepare data for ML

In [16]:
#Transform to a Data Frame for input to Machine Learing

#stage 1: as before
#stage 2: as before 
                                  
#stage 3: create the model 
classifier_gb = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3, elasticNetParam=0.8)

#regroup the stages in a pipeline
pipeline_gb = Pipeline(stages=[stringIndexer, vectorAssembler, classifier_gb])

# split data
#(data_train, data_test) = iris_df.randomSplit([0.75, 0.25])

#### Perform Machine Learning

In [17]:
# apply the pipeline to apply the transformation and train the model 
model_gb = pipeline_gb.fit(data_train)

# make predictions by transforming the features dataframe into predictions dataframe
predictions_gb = model_gb.transform(data_test)

print("model info : ",model_gb.stages[-1])

predictions_gb.select("prediction","label", "features").show()

#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
accuracy_gb = evaluator.evaluate(predictions_gb)    

print("Test accuracy = ", accuracy_gb*100, "%")

#Draw a confusion matrix
predictions_gb.groupBy("label","prediction").count().show()

model info :  LogisticRegressionModel: uid=LogisticRegression_d6b9261ecd53, numClasses=3, numFeatures=4
+----------+-----+-----------------+
|prediction|label|         features|
+----------+-----+-----------------+
|       2.0|  2.0|[4.4,2.9,1.4,0.2]|
|       2.0|  2.0|[4.6,3.4,1.4,0.3]|
|       2.0|  2.0|[4.8,3.0,1.4,0.1]|
|       2.0|  2.0|[4.8,3.1,1.6,0.2]|
|       2.0|  2.0|[4.9,3.6,1.4,0.1]|
|       2.0|  2.0|[5.0,3.3,1.4,0.2]|
|       2.0|  2.0|[5.1,3.3,1.7,0.5]|
|       2.0|  2.0|[5.1,3.5,1.4,0.2]|
|       2.0|  2.0|[5.1,3.8,1.5,0.3]|
|       2.0|  2.0|[5.1,3.8,1.6,0.2]|
|       2.0|  2.0|[5.2,3.4,1.4,0.2]|
|       2.0|  2.0|[5.4,3.4,1.7,0.2]|
|       2.0|  2.0|[5.4,3.9,1.3,0.4]|
|       1.0|  0.0|[5.5,2.3,4.0,1.3]|
|       2.0|  2.0|[5.5,4.2,1.4,0.2]|
|       1.0|  0.0|[5.6,2.5,3.9,1.1]|
|       1.0|  0.0|[5.7,2.8,4.5,1.3]|
|       1.0|  0.0|[6.0,2.2,4.0,1.0]|
|       1.0|  0.0|[6.1,2.8,4.7,1.2]|
|       1.0|  0.0|[6.2,2.2,4.5,1.5]|
+----------+-----+-----------------+
only sho